### Create new price list
(С всеми новыми тарифами в Combined, за исключением новых стран.)<br>
!!! Доработать для добавления новых стран !!!
<br>
Пока новые страны выявялть на первом этапе при анализе данных
<br>
Последнее обновление 12.07.2019 для изменений тарифов P4

In [4]:
import numpy as np
import pandas as pd
import pandasql as ps
from pandas import Series, DataFrame
from os.path import join

load_folder = r'c:/Users/balob/Documents/GITLAB/RB_BD/DATA/'
file_result = 'df_combined_with_new.csv'
file_s1_s2_tariffs_compare = 'df_s1_s2_tariffs_compare.csv'

file_al = 'DMI.dmi_allowed_list_export_Thu_Apr_30_2020_after.csv'

file_tagid_list_gsma = 'TADIG_LIST_GSMA_190227.csv'
file_tagid_list_mccmnc_ir21 = 'tadig_mccmnc_ir21.csv'

eur_usd = 1.15

### Подготовка данных

In [1]:
# Из базы данных OCS выгружаем список TADIG с NETWORK_ID

import pandas as pd
import pyodbc 
#Connect to OCSDBREP1 (BSS)
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=172.18.11.82;DATABASE=10028;UID=BSS;PWD=iKQVm40AZAmyRaw72LeY')

sql_srt=\
'''
SELECT COUNTRY_NAME,NETWORK_NAME,TADIG_CODE,NETWORK_ID,MCCMNC FROM
(
SELECT DISTINCT c.COUNTRY_NAME,n.NETWORK_NAME,t.TADIG_CODE,t.NETWORK_ID
,np.MCC + IIF(LEN(MNC)=1,'0' + MNC,MNC) MCCMNC
,ROW_NUMBER() OVER(PARTITION BY c.COUNTRY_NAME,n.NETWORK_NAME,t.TADIG_CODE ORDER BY t.TADIG_CODE DESC) AS ROW_ID
FROM RDB_TADIG_CODES t
LEFT JOIN RDB_NETWORKS n ON t.network_id = n.network_id
LEFT JOIN RDB_COUNTRIES c ON n.country_id = c.country_id
LEFT JOIN RDB_NETWORK_IMSI_PREFIXES np ON t.network_id = np.network_id
) t
WHERE ROW_ID=1
'''

df_tadig_mccmnc = pd.read_sql_query(sql_srt, cnxn)
    
###################################################
# Adjustments

# Remove YUGPM
df_tadig_mccmnc.drop(
    df_tadig_mccmnc[(df_tadig_mccmnc.TADIG_CODE=='YUGPM') & (df_tadig_mccmnc.NETWORK_ID==623)].index, inplace=True)
# c:\W_DATA_ROAM\ДОКУМЕНТАЦИЯ\PRICES\Partner_TURTK_YUGPM_not_discounted.msg
###################################################

df_tadig = df_tadig_mccmnc[['COUNTRY_NAME','NETWORK_NAME','TADIG_CODE','NETWORK_ID']].drop_duplicates()

df_tadig_mccmnc = df_tadig_mccmnc[['NETWORK_ID','MCCMNC']].drop_duplicates()

display(df_tadig.head())

cnxn.close()

,COUNTRY_NAME,NETWORK_NAME,TADIG_CODE,NETWORK_ID
0,Afghanistan,Afghan Wireless Communication Company,AFGAW,4.0
1,Afghanistan,Areeba/MTN,AFGAR,5.0
2,Afghanistan,Etisalat,AFG55,6.0
3,Afghanistan,Etisalat,AFGEA,6.0
4,Afghanistan,Roshan/TDCA,AFGK9,7.0


In [18]:
###############################################
# Опция №1
###############################################

# # Загружаем новые тарифы Partner и P4 из файлов

# '''df_partner = pd.read_csv(join(load_folder,'Partner_1_1_2019_for_tariffs.csv'),sep=';')
# print('Тарифы Partner:')
# display(df_partner.head())'''

# '''df_p4 = pd.read_csv(join(load_folder,'P4_1_1_2019_for_tariffs.csv'),sep=';')
# #df_p4[['MOC','MTC','SMS','MB']] = df_p4[['MOC','MTC','SMS','MB']]*eur_usd
# print('Тарифы P4:')
# display(df_p4.head())'''

# df_mb = pd.read_csv(join(load_folder,'NEW_TARIFFS.csv'),sep=';')
# print('Тарифы MB:')
# display(df_mb.head())

Тарифы MB:


,Country,Partner Name,TADIG,MOC,MTC,SMS,MB,Voice_Increment_SEC_S1,Data_Increment_KB_S1
0,HK,3HK,HKGH3,0.013,0.013,0.13,0.003150,1,1
1,Macau,3 Macau,MACHT,0.130,0.130,0.13,0.005040,1,1
2,CHINA,UNICOM,CHNCU,0.260,0.260,0.26,0.006301,1,1
3,UAE,Du,AREDU,0.260,0.260,0.26,0.011971,1,1
4,USA,AT&T,USACG,0.260,0.260,0.26,0.005040,1,1


In [2]:
###############################################
# Опция №2 (ИСПОЛЬЗОВАТЬ)
###############################################

# Загружаем тарифы Partner и P4 из базы данных OCS и RoamDb

# Из базы данных OCS загружаются тарифы
# Из базы данных RoamDb загружаются Voice_Increment_SEC, Data_Increment_KB, support4G
# Объединение выполняется по TADIG

# Указать в запросоах OCS roaming_plan_id
# Указать в запросоах RoamDB tarifflistid

# Connect to OCSDBREP1 (BSS)

cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=172.18.11.82;DATABASE=10028;UID=BSS;PWD=iKQVm40AZAmyRaw72LeY')

###############################################
# OCS DB Partner cost (roaming_plan_id=267)
sql_srt=\
'''
SELECT
c.country_name AS COUNTRY
,n.network_name AS Operator
,td.tadig_code AS TADIG
,mn.mcc + CASE WHEN LEN(mn.mnc)=1 THEN '0'+ mn.mnc ELSE mn.mnc END AS MCCMNC
,rpr.mo_call_rate AS MOC
,rpr.mt_call_rate AS MTC
,rpr.mo_sms_rate AS SMS
,rpr.mt_sms_rate
,rpr.data_rate AS MB
,CASE WHEN rpr.currency_id=1 THEN 'USD' WHEN rpr.currency_id=2 THEN 'EUR' END AS Currency
,CASE WHEN rpr.sponsor_id=1 THEN 'S1' WHEN rpr.sponsor_id=2 THEN 'S2' END AS Sponsor
,rpr.START_DATE
,rpr.NETWORK_ID
,rpr.DailyCapLimit
FROM ROAMING_PLAN_RULES rpr
LEFT JOIN RDB_NETWORKS n
ON rpr.network_id=n.network_id
LEFT JOIN RDB_COUNTRIES c
ON n.country_id=c.country_id
LEFT JOIN RDB_TADIG_CODES td
ON rpr.network_id=td.network_id
LEFT JOIN RDB_NETWORK_IMSI_PREFIXES mn
ON rpr.network_id=mn.network_id
WHERE rpr.roaming_plan_id=267 --Partner
AND rpr.end_date IS NULL
AND rpr.is_discounted = 1
'''
df_partner_cost = pd.read_sql_query(sql_srt, cnxn)

###############################################
# OCS DB P4 Cost (roaming_plan_id=268) with convertion to USD from EUR
sql_srt=\
'''
SELECT
c.country_name AS COUNTRY
,n.network_name AS Operator
,td.tadig_code AS TADIG
,mn.mcc + CASE WHEN LEN(mn.mnc)=1 THEN '0'+ mn.mnc ELSE mn.mnc END AS MCCMNC
,CASE WHEN rpr.currency_id=1 THEN rpr.mo_call_rate WHEN rpr.currency_id=2 THEN rpr.mo_call_rate*1.2 END MOC
,CASE WHEN rpr.currency_id=1 THEN rpr.mt_call_rate WHEN rpr.currency_id=2 THEN rpr.mt_call_rate*1.2 END MTC
,CASE WHEN rpr.currency_id=1 THEN rpr.mo_sms_rate WHEN rpr.currency_id=2 THEN rpr.mo_sms_rate*1.2 END SMS
,CASE WHEN rpr.currency_id=1 THEN rpr.mt_sms_rate WHEN rpr.currency_id=2 THEN rpr.mt_sms_rate*1.2 END mt_sms_rate
,CASE WHEN rpr.currency_id=1 THEN rpr.data_rate WHEN rpr.currency_id=2 THEN rpr.data_rate*1.2 END MB
,'USD' Currency
,CASE WHEN rpr.sponsor_id=1 THEN 'S1' WHEN rpr.sponsor_id=2 THEN 'S2' END AS Sponsor
,rpr.START_DATE
,rpr.NETWORK_ID
,rpr.DailyCapLimit
FROM ROAMING_PLAN_RULES rpr
LEFT JOIN RDB_NETWORKS n
ON rpr.network_id=n.network_id
LEFT JOIN RDB_COUNTRIES c
ON n.country_id=c.country_id
LEFT JOIN RDB_TADIG_CODES td
ON rpr.network_id=td.network_id
LEFT JOIN RDB_NETWORK_IMSI_PREFIXES mn
ON rpr.network_id=mn.network_id
WHERE rpr.roaming_plan_id=268 --P4
AND rpr.end_date IS NULL
AND rpr.is_discounted = 1
'''
df_p4_cost = pd.read_sql_query(sql_srt, cnxn)

###############################################
# OCS DB MB cost (roaming_plan_id=329)
sql_srt=\
'''
SELECT
c.country_name AS COUNTRY
,n.network_name AS Operator
,td.tadig_code AS TADIG
,mn.mcc + CASE WHEN LEN(mn.mnc)=1 THEN '0'+ mn.mnc ELSE mn.mnc END AS MCCMNC
,rpr.mo_call_rate AS MOC
,rpr.mt_call_rate AS MTC
,rpr.mo_sms_rate AS SMS
,rpr.mt_sms_rate
,rpr.data_rate AS MB
,CASE WHEN rpr.currency_id=1 THEN 'USD' WHEN rpr.currency_id=2 THEN 'EUR' END AS Currency
,CASE WHEN rpr.sponsor_id=1 THEN 'S1' WHEN rpr.sponsor_id=2 THEN 'S2' WHEN rpr.sponsor_id=4 THEN 'S4' END AS Sponsor
,rpr.START_DATE
,rpr.NETWORK_ID
,rpr.DailyCapLimit
FROM ROAMING_PLAN_RULES rpr
LEFT JOIN RDB_NETWORKS n
ON rpr.network_id=n.network_id
LEFT JOIN RDB_COUNTRIES c
ON n.country_id=c.country_id
LEFT JOIN RDB_TADIG_CODES td
ON rpr.network_id=td.network_id
LEFT JOIN RDB_NETWORK_IMSI_PREFIXES mn
ON rpr.network_id=mn.network_id
WHERE rpr.roaming_plan_id=329 --MB
AND rpr.end_date IS NULL
AND rpr.is_discounted = 1
'''
df_mb_cost = pd.read_sql_query(sql_srt, cnxn)

###############################################
# OCS DB SMART cost (roaming_plan_id=343)
sql_srt=\
'''
SELECT
c.country_name AS COUNTRY
,n.network_name AS Operator
,td.tadig_code AS TADIG
,mn.mcc + CASE WHEN LEN(mn.mnc)=1 THEN '0'+ mn.mnc ELSE mn.mnc END AS MCCMNC
,rpr.mo_call_rate AS MOC
,rpr.mt_call_rate AS MTC
,rpr.mo_sms_rate AS SMS
,rpr.mt_sms_rate
,rpr.data_rate AS MB
,CASE WHEN rpr.currency_id=1 THEN 'USD' WHEN rpr.currency_id=2 THEN 'EUR' END AS Currency
,CASE WHEN rpr.sponsor_id=1 THEN 'S1' WHEN rpr.sponsor_id=2 THEN 'S2' WHEN rpr.sponsor_id=4 THEN 'S4' END AS Sponsor
,rpr.START_DATE
,rpr.NETWORK_ID
,rpr.DailyCapLimit
FROM ROAMING_PLAN_RULES rpr
LEFT JOIN RDB_NETWORKS n
ON rpr.network_id=n.network_id
LEFT JOIN RDB_COUNTRIES c
ON n.country_id=c.country_id
LEFT JOIN RDB_TADIG_CODES td
ON rpr.network_id=td.network_id
LEFT JOIN RDB_NETWORK_IMSI_PREFIXES mn
ON rpr.network_id=mn.network_id
WHERE rpr.roaming_plan_id=343 --SMART
AND rpr.end_date IS NULL
AND rpr.is_discounted = 1
'''
df_smart_cost = pd.read_sql_query(sql_srt, cnxn)

###############################################
# OCS DB Orange cost (roaming_plan_id=267)
sql_srt=\
'''
SELECT
c.country_name AS COUNTRY
,n.network_name AS Operator
,td.tadig_code AS TADIG
,mn.mcc + CASE WHEN LEN(mn.mnc)=1 THEN '0'+ mn.mnc ELSE mn.mnc END AS MCCMNC
,rpr.mo_call_rate AS MOC
,rpr.mt_call_rate AS MTC
,rpr.mo_sms_rate AS SMS
,rpr.mt_sms_rate
,rpr.data_rate AS MB
,CASE WHEN rpr.currency_id=1 THEN 'USD' WHEN rpr.currency_id=2 THEN 'EUR' END AS Currency
,CASE WHEN rpr.sponsor_id=6 THEN 'S6' END AS Sponsor
,rpr.START_DATE
,rpr.NETWORK_ID
,rpr.DailyCapLimit
FROM ROAMING_PLAN_RULES rpr
LEFT JOIN RDB_NETWORKS n
ON rpr.network_id=n.network_id
LEFT JOIN RDB_COUNTRIES c
ON n.country_id=c.country_id
LEFT JOIN RDB_TADIG_CODES td
ON rpr.network_id=td.network_id
LEFT JOIN RDB_NETWORK_IMSI_PREFIXES mn
ON rpr.network_id=mn.network_id
WHERE rpr.roaming_plan_id=368 --Orange Cost 2019
AND rpr.end_date IS NULL
AND rpr.is_discounted = 1
'''
df_orange_cost = pd.read_sql_query(sql_srt, cnxn)

###############################################
# OCS DB JT cost (roaming_plan_id=372)
sql_srt=\
'''
SELECT
c.country_name AS COUNTRY
,n.network_name AS Operator
,td.tadig_code AS TADIG
,mn.mcc + CASE WHEN LEN(mn.mnc)=1 THEN '0'+ mn.mnc ELSE mn.mnc END AS MCCMNC
,rpr.mo_call_rate AS MOC
,rpr.mt_call_rate AS MTC
,rpr.mo_sms_rate AS SMS
,rpr.mt_sms_rate
,rpr.data_rate AS MB
,CASE WHEN rpr.currency_id=1 THEN 'USD' WHEN rpr.currency_id=2 THEN 'EUR' END AS Currency
,CASE WHEN rpr.sponsor_id=1 THEN 'S1' WHEN rpr.sponsor_id=2 THEN 'S2' WHEN rpr.sponsor_id=4 THEN 'S4' WHEN rpr.sponsor_id=8 THEN 'S8' END AS Sponsor
,rpr.START_DATE
,rpr.NETWORK_ID
,rpr.DailyCapLimit
FROM ROAMING_PLAN_RULES rpr
LEFT JOIN RDB_NETWORKS n
ON rpr.network_id=n.network_id
LEFT JOIN RDB_COUNTRIES c
ON n.country_id=c.country_id
LEFT JOIN RDB_TADIG_CODES td
ON rpr.network_id=td.network_id
LEFT JOIN RDB_NETWORK_IMSI_PREFIXES mn
ON rpr.network_id=mn.network_id
WHERE rpr.roaming_plan_id=372 --JT
AND rpr.end_date IS NULL
AND rpr.is_discounted = 1
'''
df_jt_cost = pd.read_sql_query(sql_srt, cnxn)

cnxn.close()

###############################################
# Connect to RoamDB

cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=172.19.11.59;DATABASE=RoamDB;UID=roamdb;PWD=roamdbpasswd')

###############################################
# RoamDB Partner cost
sql_srt=\
'''
SELECT t.outgoingvoiceincrement_SEC AS Voice_Increment_SEC
,t.dataincrement_KB AS Data_Increment_KB
,t.support4G,tc.value AS TADIG FROM Tariffs t
INNER JOIN TadigCodes tc
ON t.OperatorID = tc.OperatorID
WHERE t.tarifflistid = 35 --'Partner Mobile Costs 2020'
'''
df_partner_2019 = pd.read_sql_query(sql_srt, cnxn)

###############################################
# RoamDB P4 Cost
sql_srt=\
'''
SELECT t.outgoingvoiceincrement_SEC AS Voice_Increment_SEC
,t.dataincrement_KB AS Data_Increment_KB
,t.support4G,tc.value AS TADIG FROM Tariffs t
INNER JOIN TadigCodes tc
ON t.OperatorID = tc.OperatorID
WHERE t.tarifflistid = 34 --'P4 Mobile Costs 2020'
'''
df_p4_2019 = pd.read_sql_query(sql_srt, cnxn)

###############################################
# RoamDB MB Cost
sql_srt=\
'''
SELECT t.outgoingvoiceincrement_SEC AS Voice_Increment_SEC
,t.dataincrement_KB AS Data_Increment_KB
,t.support4G,tc.value AS TADIG FROM Tariffs t
INNER JOIN TadigCodes tc
ON t.OperatorID = tc.OperatorID
WHERE t.tarifflistid = 36 --'MB Mobile Costs 2020'
'''
df_mb_2019 = pd.read_sql_query(sql_srt, cnxn)

###############################################
# RoamDB SMART Cost (tarifflistid=32)
sql_srt=\
'''
SELECT t.outgoingvoiceincrement_SEC AS Voice_Increment_SEC
,t.dataincrement_KB AS Data_Increment_KB
,t.support4G,tc.value AS TADIG FROM Tariffs t
INNER JOIN TadigCodes tc
ON t.OperatorID = tc.OperatorID
WHERE t.tarifflistid = 38 --'MB Mobile Costs 2020'
'''
df_smart_2019 = pd.read_sql_query(sql_srt, cnxn)

###############################################
# RoamDB Orange cost
sql_srt=\
'''
SELECT t.outgoingvoiceincrement_SEC AS Voice_Increment_SEC
,t.dataincrement_KB AS Data_Increment_KB
,t.support4G,tc.value AS TADIG FROM Tariffs t
INNER JOIN TadigCodes tc
ON t.OperatorID = tc.OperatorID
WHERE t.tarifflistid = 39 --'Orange Mobile Costs 2020'
'''
df_orange_2019 = pd.read_sql_query(sql_srt, cnxn)

###############################################
# RoamDB JT Cost (tarifflistid=33)
sql_srt=\
'''
SELECT t.outgoingvoiceincrement_SEC AS Voice_Increment_SEC
,t.dataincrement_KB AS Data_Increment_KB
,t.support4G,tc.value AS TADIG FROM Tariffs t
INNER JOIN TadigCodes tc
ON t.OperatorID = tc.OperatorID
WHERE t.tarifflistid = 37 --'JT Mobile Costs 2020'
'''
df_jt_2019 = pd.read_sql_query(sql_srt, cnxn)

cnxn.close()

df_partner=pd.merge(df_partner_cost, df_partner_2019, how='left', on='TADIG')

df_p4=pd.merge(df_p4_cost, df_p4_2019, how='left', on='TADIG')

df_mb=pd.merge(df_mb_cost, df_mb_2019, how='left', on='TADIG')

df_smart=pd.merge(df_smart_cost, df_smart_2019, how='left', on='TADIG')

df_orange=pd.merge(df_orange_cost, df_orange_2019, how='left', on='TADIG')

df_jt=pd.merge(df_jt_cost, df_jt_2019, how='left', on='TADIG')

print('\nPartner tariffs OCS')
display(df_partner.head(3))

print('\nP4 tariffs OCS')
display(df_p4.head(3))

print('\nMB tariffs OCS')
display(df_mb.head(3))

print('\nSMART tariffs OCS')
display(df_smart.head(3))

print('\nOrange tariffs OCS')
display(df_orange.head(3))

print('\nJT tariffs OCS')
display(df_jt.head(3))


Partner tariffs OCS


,COUNTRY,Operator,TADIG,MCCMNC,MOC,MTC,SMS,mt_sms_rate,MB,Currency,Sponsor,START_DATE,NETWORK_ID,DailyCapLimit,Voice_Increment_SEC,Data_Increment_KB,support4G
0,Albania,TELEKOM ALBANIA SH A,ALBAM,27601,0.2600,0.0,0.0300,0.0,0.0090,USD,S1,2020-01-01,9.0,None,1.0,1.0,False
1,Albania,Vodafone,ALBVF,27602,0.0300,0.0,0.0060,0.0,0.0024,USD,S1,2020-01-01,12.0,None,1.0,1.0,True
2,Anguilla,Cable and Wireless,AIACW,365840,0.1606,0.0,0.0128,0.0,0.0257,USD,S1,2019-01-01,20.0,None,60.0,1.0,False



P4 tariffs OCS


,COUNTRY,Operator,TADIG,MCCMNC,MOC,MTC,SMS,mt_sms_rate,MB,Currency,Sponsor,START_DATE,NETWORK_ID,DailyCapLimit,Voice_Increment_SEC,Data_Increment_KB,support4G
0,Albania,TELEKOM ALBANIA SH A,ALBAM,27601,0.51635,0.0,0.0598,0.0,0.04600,USD,S2,2020-01-01,9.0,None,1.0,1.0,True
1,Albania,Eagle Mobile,ALBEM,27603,0.44850,0.0,0.0460,0.0,0.04485,USD,S2,2020-01-01,10.0,None,60.0,1.0,True
2,Australia,Singtel Optus,AUSOP,50590,0.06325,0.0,0.0092,0.0,0.00920,USD,S2,2020-01-01,45.0,None,60.0,1.0,True



MB tariffs OCS


...

,COUNTRY,Operator,TADIG,MCCMNC,MOC,MTC,SMS,mt_sms_rate,MB,Currency,Sponsor,START_DATE,NETWORK_ID,DailyCapLimit,Voice_Increment_SEC,Data_Increment_KB,support4G
0,Afghanistan,Afghan Wireless Communication Company,AFGAW,41201,NaN,0.1725,0.11500,0.0,0.11500,USD,None,2020-01-21,4.0,NaN,60.0,1.0,False
1,Afghanistan,Etisalat,AFG55,41250,NaN,0.0000,0.01725,0.0,0.01725,USD,None,2020-01-21,6.0,NaN,60.0,1.0,False
2,Afghanistan,Etisalat,AFGEA,41250,NaN,0.0000,0.01725,0.0,0.01725,USD,None,2020-01-21,6.0,NaN,60.0,1.0,False



Orange tariffs OCS


,COUNTRY,Operator,TADIG,MCCMNC,MOC,MTC,SMS,mt_sms_rate,MB,Currency,Sponsor,START_DATE,NETWORK_ID,DailyCapLimit,Voice_Increment_SEC,Data_Increment_KB,support4G
0,Albania,TELEKOM ALBANIA SH A,ALBAM,27601,0.44850,0.0,0.007475,0.0,0.01495,USD,S6,2020-01-01,9.0,None,1.0,1.0,True
1,Albania,Vodafone,ALBVF,27602,0.22425,0.0,0.007475,0.0,0.01495,USD,S6,2020-01-01,12.0,None,1.0,1.0,True
2,Algeria,ATM Mobils,DZAA1,60301,0.59800,0.0,0.014950,0.0,0.01495,USD,S6,2020-01-01,13.0,None,1.0,1.0,False



JT tariffs OCS


,COUNTRY,Operator,TADIG,MCCMNC,MOC,MTC,SMS,mt_sms_rate,MB,Currency,Sponsor,START_DATE,NETWORK_ID,DailyCapLimit,Voice_Increment_SEC,Data_Increment_KB,support4G
0,Afghanistan,Areeba/MTN,AFGAR,41240,1.0164,0.0,0.1236,0.0,0.1044,USD,S8,2019-12-01,5.0,None,None,1.0,False
1,Afghanistan,Etisalat,AFG55,41250,0.7956,0.0,0.2880,0.0,0.2976,USD,S8,2019-12-01,6.0,None,None,1.0,False
2,Afghanistan,Etisalat,AFGEA,41250,0.7956,0.0,0.2880,0.0,0.2976,USD,S8,2019-12-01,6.0,None,None,1.0,False


In [5]:
# Загружаем Combined Allowed list, объединяем по TADIG_CODE с df_tadig из OCS для добавления NETWORK_ID
# Вносим необходимые изменения в Allowed list

df_al=pd.read_csv(join(load_folder, file_al),usecols=['Sponsor','PLMN code'])
df_al.rename({'PLMN code':'TADIG_CODE_AL','Sponsor':'SPONSOR'},inplace=True,axis=1)
df_al = df_al[df_al.SPONSOR.isin(['Partner_Combined','P4_Combined','MB_Combined','SMART_Combined','S8_Combined'])]
df_al = df_al.replace({'Partner_Combined':'S1','P4_Combined':'S2','MB_Combined':'S4','SMART_Combined':'S5','S8_Combined':'S8'})

###################################################
# Adjustments

# Заменить операторов

# change_operators= {'MACHT':'S4',
#                    'THAWN':'S4',
#                    'THACO':'S4',}

# for change_operator in change_operators:
#     df_al.loc[df_al.TADIG_CODE_AL == change_operator, 'SPONSOR'] = change_operators[change_operator]

# Доавить операторов в Combined

# new_operators = [{'SPONSOR':'S4','TADIG_CODE_AL':'AREDU'},
#                  {'SPONSOR':'S4','TADIG_CODE_AL':'SAUVG'},]

# df_al = df_al.append(new_operators)

# # Удалить операторов из Combined

# remove_operators = ['THADT','THAWP']

# df_al.drop(df_al[df_al.TADIG_CODE_AL.isin(remove_operators)].index, inplace=True)

###################################################

df_al = pd.merge(df_al, df_tadig[['TADIG_CODE','NETWORK_ID']], how='left',left_on='TADIG_CODE_AL',right_on='TADIG_CODE')
df_al.drop('TADIG_CODE', inplace=True,axis=1)

print('Allowed list:')
display(df_al.head(3))

Allowed list:


,SPONSOR,TADIG_CODE_AL,NETWORK_ID
0,S2,HRVT2,225.0
1,S2,CZEET,234.0
2,S1,GBRAJ,998.0


In [6]:
# Check for different Sponsors in the same country

df_temp = df_al.copy()
df_temp['COUNTRY'] = df_al.TADIG_CODE_AL.str.slice(0,3)

s1_temp = df_temp.groupby('COUNTRY')['SPONSOR'].nunique()
s1_temp[s1_temp.values > 1]

Series([], Name: SPONSOR, dtype: int64)

In [7]:
# Объединяем тарифы всех спонсоров в один DF и объединяем с Allowed list
# Полученный DF выгружается в файл 'df_s1_s2_tariffs_compare.csv'

# Объединить S1 и S2
df_s1_s2_temp = pd.merge(
    df_partner[['MOC','MTC','SMS','MB','DailyCapLimit','NETWORK_ID','Voice_Increment_SEC','Data_Increment_KB','START_DATE','support4G']],
    df_p4[['MOC','MTC','SMS','MB','DailyCapLimit','NETWORK_ID','Voice_Increment_SEC','Data_Increment_KB','START_DATE','support4G']],
    how='outer', on=['NETWORK_ID'], suffixes=('_S1','_S2')).drop_duplicates()

# Объединить S1/S2 и S4
df_s1_s2_s4_temp = pd.merge(
    df_s1_s2_temp,
    df_mb[['MOC','MTC','SMS','MB','DailyCapLimit','NETWORK_ID','Voice_Increment_SEC','Data_Increment_KB','START_DATE','support4G']],
    how='outer',on=['NETWORK_ID']).drop_duplicates()

df_s1_s2_s4_temp.rename({'MOC':'MOC_S4','MTC':'MTC_S4','SMS':'SMS_S4','MB':'MB_S4','DailyCapLimit':'DailyCapLimit_S4'
                         ,'Voice_Increment_SEC':'Voice_Increment_SEC_S4','Data_Increment_KB':'Data_Increment_KB_S4'
                         ,'START_DATE':'START_DATE_S4','support4G':'support4G_S4'},
                         axis=1,inplace=True)

# Объединить S1/S2/S4 и S5
df_s1_s2_s4_s5_temp = pd.merge(
    df_s1_s2_s4_temp,
    df_smart[['MOC','MTC','SMS','MB','DailyCapLimit','NETWORK_ID','Voice_Increment_SEC','Data_Increment_KB','START_DATE','support4G']],
    how='outer',on=['NETWORK_ID']).drop_duplicates()

df_s1_s2_s4_s5_temp.rename({'MOC':'MOC_S5','MTC':'MTC_S5','SMS':'SMS_S5','MB':'MB_S5','DailyCapLimit':'DailyCapLimit_S5'
                         ,'Voice_Increment_SEC':'Voice_Increment_SEC_S5','Data_Increment_KB':'Data_Increment_KB_S5'
                         ,'START_DATE':'START_DATE_S5','support4G':'support4G_S5'},
                         axis=1,inplace=True)


# Объединить S1/S2/S4/S5 + S8
df_s1_s2_s4_s5_temp = pd.merge(
    df_s1_s2_s4_s5_temp,
    df_jt[['MOC','MTC','SMS','MB','DailyCapLimit','NETWORK_ID','Voice_Increment_SEC','Data_Increment_KB','START_DATE','support4G']],
    how='outer',on=['NETWORK_ID']).drop_duplicates()

df_s1_s2_s4_s5_temp.rename({'MOC':'MOC_S8','MTC':'MTC_S8','SMS':'SMS_S8','MB':'MB_S8','DailyCapLimit':'DailyCapLimit_S8'
                         ,'Voice_Increment_SEC':'Voice_Increment_SEC_S8','Data_Increment_KB':'Data_Increment_KB_S8'
                         ,'START_DATE':'START_DATE_S8','support4G':'support4G_S8'},
                         axis=1,inplace=True)

# Объединить S1/S2/S4/S5/S8 + S6
df_s1_s2_s4_s5_temp = pd.merge(
    df_s1_s2_s4_s5_temp,
    df_orange[['MOC','MTC','SMS','MB','DailyCapLimit','NETWORK_ID','Voice_Increment_SEC','Data_Increment_KB','START_DATE','support4G']],
    how='outer',on=['NETWORK_ID']).drop_duplicates()

df_s1_s2_s4_s5_temp.rename({'MOC':'MOC_S6','MTC':'MTC_S6','SMS':'SMS_S6','MB':'MB_S6','DailyCapLimit':'DailyCapLimit_S6'
                         ,'Voice_Increment_SEC':'Voice_Increment_SEC_S6','Data_Increment_KB':'Data_Increment_KB_S6'
                         ,'START_DATE':'START_DATE_S6','support4G':'support4G_S6'},
                         axis=1,inplace=True)


df_s1_s2 = pd.merge(df_s1_s2_s4_s5_temp, df_tadig, how='left', on='NETWORK_ID')

df_s1_s2_sponsor = pd.merge(df_s1_s2, df_al, how='outer', on='NETWORK_ID')
df_s1_s2_sponsor['COUNTRY_SPONSOR'] = df_s1_s2_sponsor.groupby('COUNTRY_NAME')['SPONSOR'].transform('any')

# Результирующий файл

f_list = ['COUNTRY_NAME','NETWORK_NAME','TADIG_CODE',
          'MOC_S1','MTC_S1','SMS_S1','MB_S1','DailyCapLimit_S1','Voice_Increment_SEC_S1','Data_Increment_KB_S1','START_DATE_S1','support4G_S1',
          'MOC_S2','MTC_S2','SMS_S2','MB_S2','DailyCapLimit_S2','Voice_Increment_SEC_S2','Data_Increment_KB_S2','START_DATE_S2','support4G_S2',
          'MOC_S4','MTC_S4','SMS_S4','MB_S4','DailyCapLimit_S4','Voice_Increment_SEC_S4','Data_Increment_KB_S4','START_DATE_S4','support4G_S4',
          'MOC_S5','MTC_S5','SMS_S5','MB_S5','DailyCapLimit_S5','Voice_Increment_SEC_S5','Data_Increment_KB_S5','START_DATE_S5','support4G_S5',
          'MOC_S6','MTC_S6','SMS_S6','MB_S6','DailyCapLimit_S6','Voice_Increment_SEC_S6','Data_Increment_KB_S6','START_DATE_S6','support4G_S6',
          'MOC_S8','MTC_S8','SMS_S8','MB_S8','DailyCapLimit_S8','Voice_Increment_SEC_S8','Data_Increment_KB_S8','START_DATE_S8','support4G_S8',
          'SPONSOR','COUNTRY_SPONSOR','TADIG_CODE_AL','NETWORK_ID']
df_s1_s2_sponsor = df_s1_s2_sponsor[f_list].drop_duplicates()

df_s1_s2_sponsor.to_csv(join(load_folder, file_s1_s2_tariffs_compare),index=False)

print('\nТарифы всех спонсоров с привязкой к Allowed list:')
df_s1_s2_sponsor.head(3)


Тарифы всех спонсоров с привязкой к Allowed list:


,COUNTRY_NAME,NETWORK_NAME,TADIG_CODE,MOC_S1,MTC_S1,SMS_S1,MB_S1,DailyCapLimit_S1,Voice_Increment_SEC_S1,Data_Increment_KB_S1,...,MB_S8,DailyCapLimit_S8,Voice_Increment_SEC_S8,Data_Increment_KB_S8,START_DATE_S8,support4G_S8,SPONSOR,COUNTRY_SPONSOR,TADIG_CODE_AL,NETWORK_ID
0,Albania,TELEKOM ALBANIA SH A,ALBAM,0.2600,0.0,0.0300,0.0090,None,1.0,1.0,...,0.0180,None,None,1.0,2019-12-01,False,NaN,True,NaN,9.0
1,Albania,Vodafone,ALBVF,0.0300,0.0,0.0060,0.0024,None,1.0,1.0,...,0.0204,None,None,1.0,2019-12-01,True,S1,True,ALBVF,12.0
2,Anguilla,Cable and Wireless,AIACW,0.1606,0.0,0.0128,0.0257,None,60.0,1.0,...,0.1260,None,None,1.0,2019-12-01,True,S1,True,AIACW,20.0


In [8]:
# Добавляем MCCMNC

df_s1_s2_sponsor_mccmnc = pd.merge(df_s1_s2_sponsor, df_tadig_mccmnc, how='left',on='NETWORK_ID')
df_s1_s2_sponsor_mccmnc.head(3)

,COUNTRY_NAME,NETWORK_NAME,TADIG_CODE,MOC_S1,MTC_S1,SMS_S1,MB_S1,DailyCapLimit_S1,Voice_Increment_SEC_S1,Data_Increment_KB_S1,...,DailyCapLimit_S8,Voice_Increment_SEC_S8,Data_Increment_KB_S8,START_DATE_S8,support4G_S8,SPONSOR,COUNTRY_SPONSOR,TADIG_CODE_AL,NETWORK_ID,MCCMNC
0,Albania,TELEKOM ALBANIA SH A,ALBAM,0.2600,0.0,0.0300,0.0090,None,1.0,1.0,...,None,None,1.0,2019-12-01,False,NaN,True,NaN,9.0,27601
1,Albania,Vodafone,ALBVF,0.0300,0.0,0.0060,0.0024,None,1.0,1.0,...,None,None,1.0,2019-12-01,True,S1,True,ALBVF,12.0,27602
2,Anguilla,Cable and Wireless,AIACW,0.1606,0.0,0.0128,0.0257,None,60.0,1.0,...,None,None,1.0,2019-12-01,True,S1,True,AIACW,20.0,365840


In [9]:
# Проверка наличия различных тарифов для одного и того же оператора.
# Операторы определяются по данным OCS

df = df_p4.groupby('NETWORK_ID')['MOC','MTC','SMS','MB'].nunique()
s1 = df[df.values > 1]
display(s1)

df = df_partner.groupby('NETWORK_ID')['MOC','MTC','SMS','MB'].nunique()
s2 = df[df.values > 1]
display(s2)

df = df_mb.groupby('NETWORK_ID')['MOC','MTC','SMS','MB'].nunique()
s4 = df[df.values > 1]
display(s4)

df = df_smart.groupby('NETWORK_ID')['MOC','MTC','SMS','MB'].nunique()
s5 = df[df.values > 1]
display(s5)

df = df_orange.groupby('NETWORK_ID')['MOC','MTC','SMS','MB'].nunique()
s6 = df[df.values > 1]
display(s6)

df = df_jt.groupby('NETWORK_ID')['MOC','MTC','SMS','MB'].nunique()
s8 = df[df.values > 1]
display(s8)

,MOC,MTC,SMS,MB
NETWORK_ID,,,,


,MOC,MTC,SMS,MB
NETWORK_ID,,,,


,MOC,MTC,SMS,MB
NETWORK_ID,,,,


,MOC,MTC,SMS,MB
NETWORK_ID,,,,


,MOC,MTC,SMS,MB
NETWORK_ID,,,,


,MOC,MTC,SMS,MB
NETWORK_ID,,,,


In [10]:
# Подготовка Combined Price Plan с MCCMNC для Пользователей
# ИСПОЛЬЗОВАТЬ

# Результат выгружается в файл file_result = 'df_combined_with_new.csv'

# Подготовка DF для S1

df1 = df_s1_s2_sponsor_mccmnc\
[['COUNTRY_NAME','NETWORK_NAME','TADIG_CODE','MCCMNC','START_DATE_S1','support4G_S1'
  ,'MOC_S1','MTC_S1','SMS_S1','MB_S1','DailyCapLimit_S1','Voice_Increment_SEC_S1','Data_Increment_KB_S1','SPONSOR','NETWORK_ID']].copy()

df1.rename({'COUNTRY_NAME':'Country','NETWORK_NAME':'Operator','TADIG_CODE':'TADIG','START_DATE_S1':'START_DATE',
            'MOC_S1':'MOC','MTC_S1':'MTC','SMS_S1':'SMS','MB_S1':'DATA (MB)','DailyCapLimit_S1':'DailyCapLimit','SPONSOR':'SPONSOR_AL',
            'Voice_Increment_SEC_S1':'Voice Increment (SEC)','Data_Increment_KB_S1':'Data Increment (KB)',
            'support4G_S1':'Support 4G'}
           ,axis=1,inplace=True)

df1['SPONSOR'] = 'S1'

# Подготовка DF для S2

df2 = df_s1_s2_sponsor_mccmnc\
[['COUNTRY_NAME','NETWORK_NAME','TADIG_CODE','MCCMNC','START_DATE_S2','support4G_S2'
  ,'MOC_S2','MTC_S2','SMS_S2','MB_S2','DailyCapLimit_S2','Voice_Increment_SEC_S2','Data_Increment_KB_S2','SPONSOR','NETWORK_ID']].copy()

df2.rename({'COUNTRY_NAME':'Country','NETWORK_NAME':'Operator','TADIG_CODE':'TADIG','START_DATE_S2':'START_DATE',
            'MOC_S2':'MOC','MTC_S2':'MTC','SMS_S2':'SMS','MB_S2':'DATA (MB)','DailyCapLimit_S2':'DailyCapLimit','SPONSOR':'SPONSOR_AL',
            'Voice_Increment_SEC_S2':'Voice Increment (SEC)','Data_Increment_KB_S2':'Data Increment (KB)',
            'support4G_S2':'Support 4G'}
           ,axis=1, inplace=True)

df2['SPONSOR'] = 'S2'

# Подготовка DF для S4

df4 = df_s1_s2_sponsor_mccmnc\
[['COUNTRY_NAME','NETWORK_NAME','TADIG_CODE','MCCMNC','START_DATE_S4','support4G_S4'
  ,'MOC_S4','MTC_S4','SMS_S4','MB_S4','DailyCapLimit_S4','Voice_Increment_SEC_S4','Data_Increment_KB_S4','SPONSOR','NETWORK_ID']].copy()

df4.rename({'COUNTRY_NAME':'Country','NETWORK_NAME':'Operator','TADIG_CODE':'TADIG','START_DATE_S4':'START_DATE',
            'MOC_S4':'MOC','MTC_S4':'MTC','SMS_S4':'SMS','MB_S4':'DATA (MB)','DailyCapLimit_S4':'DailyCapLimit','SPONSOR':'SPONSOR_AL',
            'Voice_Increment_SEC_S4':'Voice Increment (SEC)','Data_Increment_KB_S4':'Data Increment (KB)',
            'support4G_S4':'Support 4G'}
           ,axis=1, inplace=True)

df4['SPONSOR'] = 'S4'

# Подготовка DF для S5

df5 = df_s1_s2_sponsor_mccmnc\
[['COUNTRY_NAME','NETWORK_NAME','TADIG_CODE','MCCMNC','START_DATE_S5','support4G_S5'
  ,'MOC_S5','MTC_S5','SMS_S5','MB_S5','DailyCapLimit_S5','Voice_Increment_SEC_S5','Data_Increment_KB_S5','SPONSOR','NETWORK_ID']].copy()

df5.rename({'COUNTRY_NAME':'Country','NETWORK_NAME':'Operator','TADIG_CODE':'TADIG','START_DATE_S5':'START_DATE',
            'MOC_S5':'MOC','MTC_S5':'MTC','SMS_S5':'SMS','MB_S5':'DATA (MB)','DailyCapLimit_S5':'DailyCapLimit','SPONSOR':'SPONSOR_AL',
            'Voice_Increment_SEC_S5':'Voice Increment (SEC)','Data_Increment_KB_S5':'Data Increment (KB)',
            'support4G_S5':'Support 4G'}
           ,axis=1, inplace=True)

df5['SPONSOR'] = 'S5'

# Подготовка DF для S6

df6 = df_s1_s2_sponsor_mccmnc\
[['COUNTRY_NAME','NETWORK_NAME','TADIG_CODE','MCCMNC','START_DATE_S6','support4G_S6'
  ,'MOC_S6','MTC_S6','SMS_S6','MB_S6','DailyCapLimit_S6','Voice_Increment_SEC_S6','Data_Increment_KB_S6','SPONSOR','NETWORK_ID']].copy()

df6.rename({'COUNTRY_NAME':'Country','NETWORK_NAME':'Operator','TADIG_CODE':'TADIG','START_DATE_S5':'START_DATE',
            'MOC_S6':'MOC','MTC_S6':'MTC','SMS_S6':'SMS','MB_S6':'DATA (MB)','DailyCapLimit_S6':'DailyCapLimit','SPONSOR':'SPONSOR_AL',
            'Voice_Increment_SEC_S6':'Voice Increment (SEC)','Data_Increment_KB_S6':'Data Increment (KB)',
            'support4G_S6':'Support 4G'}
           ,axis=1, inplace=True)

df5['SPONSOR'] = 'S6'

# Подготовка DF для S8

df8 = df_s1_s2_sponsor_mccmnc\
[['COUNTRY_NAME','NETWORK_NAME','TADIG_CODE','MCCMNC','START_DATE_S8','support4G_S8'
  ,'MOC_S8','MTC_S8','SMS_S8','MB_S8','DailyCapLimit_S8','Voice_Increment_SEC_S8','Data_Increment_KB_S8','SPONSOR','NETWORK_ID']].copy()

df8.rename({'COUNTRY_NAME':'Country','NETWORK_NAME':'Operator','TADIG_CODE':'TADIG','START_DATE_S8':'START_DATE',
            'MOC_S8':'MOC','MTC_S8':'MTC','SMS_S8':'SMS','MB_S8':'DATA (MB)','DailyCapLimit_S8':'DailyCapLimit','SPONSOR':'SPONSOR_AL',
            'Voice_Increment_SEC_S8':'Voice Increment (SEC)','Data_Increment_KB_S8':'Data Increment (KB)',
            'support4G_S8':'Support 4G'}
           ,axis=1, inplace=True)

df8['SPONSOR'] = 'S8'

df_combined = pd.concat([df1, df2, df4, df5, df6, df8], axis=0,
                        sort=False).drop_duplicates().sort_values(by=['Country', 'Operator'])

df_combined.dropna(subset=['MOC','MTC','SMS','DATA (MB)'], how='all', inplace=True)

df_combined.MTC.fillna(0, inplace=True)

df_combined[['MOC','MTC','SMS','DATA (MB)']] = df_combined[['MOC','MTC','SMS','DATA (MB)']] * 2

###################################################
# Adjustments

# Тарифы для USA умножаются дополнииельно на 1.25
# df_combined.loc[df_combined.Country=='United States',['MOC','MTC','SMS','DATA (MB)']]\
# = df_combined[df_combined.Country=='United States'][['MOC','MTC','SMS','DATA (MB)']] * 1.25

###################################################

df_combined['COUNTRY_SPONSOR'] = df_combined.groupby('Country')['SPONSOR_AL'].transform('first')

df_combined = df_combined[df_combined.SPONSOR == df_combined.COUNTRY_SPONSOR]

df_combined.to_csv(join(load_folder, file_result), index=False)

df_combined.head(3)

,Country,Operator,TADIG,MCCMNC,START_DATE,Support 4G,MOC,MTC,SMS,DATA (MB),DailyCapLimit,Voice Increment (SEC),Data Increment (KB),SPONSOR_AL,NETWORK_ID,SPONSOR,START_DATE_S6,COUNTRY_SPONSOR
515,Afghanistan,Areeba/MTN,AFGAR,41240,2019-12-01,False,2.0328,0.0,0.2472,0.2088,None,None,1.0,NaN,5.0,S8,NaT,S8
438,Afghanistan,Etisalat,AFG55,41250,2019-12-01,False,1.5912,0.0,0.5760,0.5952,None,None,1.0,S8,6.0,S8,NaT,S8
439,Afghanistan,Etisalat,AFGEA,41250,2019-12-01,False,1.5912,0.0,0.5760,0.5952,None,None,1.0,S8,6.0,S8,NaT,S8


In [10]:
# Подготовка Partner Price Plan для Пользователей (без DailyCapLimit)

df_s1_price_plan = df_s1_s2_sponsor_mccmnc\
[['COUNTRY_NAME','NETWORK_NAME','TADIG_CODE','MCCMNC','START_DATE_S1'
  ,'MOC_S1','MTC_S1','SMS_S1','MB_S1','Voice_Increment_SEC_S1','Data_Increment_KB_S1'
  ,'support4G_S1','SPONSOR']].drop_duplicates()

df_s1_price_plan.rename({'COUNTRY_NAME':'Country','NETWORK_NAME':'Operator','TADIG_CODE':'TADIG','START_DATE_S1':'START_DATE',
            'MOC_S1':'MOC','MTC_S1':'MTC','SMS_S1':'SMS','MB_S1':'DATA (MB)',
            'Voice_Increment_SEC_S1':'Voice Increment (SEC)','Data_Increment_KB_S1':'Data Increment (KB)',
            'support4G_S1':'Support 4G'}
           ,axis=1,inplace=True)

df_s1_price_plan.dropna(subset=['MOC','MTC','SMS','DATA (MB)'], how='all', inplace=True)
df_s1_price_plan.MTC.fillna(0,inplace=True)

df_s1_price_plan[['MOC','MTC','SMS','DATA (MB)']] = df_s1_price_plan[['MOC','MTC','SMS','DATA (MB)']]*2

# df_s1_price_plan.loc[df_s1_price_plan.Country=='United States',['MOC','MTC','SMS','DATA (MB)']]\
# = df_s1_price_plan[df_s1_price_plan.Country=='United States'][['MOC','MTC','SMS','DATA (MB)']]*1.25

df_s1_price_plan.to_csv(join(load_folder,'df_s1_price_plan.csv'),index=False)

df_s1_price_plan.head()

,Country,Operator,TADIG,MCCMNC,START_DATE,MOC,MTC,SMS,DATA (MB),Voice Increment (SEC),Data Increment (KB),Support 4G,SPONSOR
0,Albania,TELEKOM ALBANIA SH A,ALBAM,27601,2020-01-01,0.5200,0.0,0.0600,0.0180,1.0,1.0,False,NaN
1,Albania,Vodafone,ALBVF,27602,2020-01-01,0.0600,0.0,0.0120,0.0048,1.0,1.0,True,S1
2,Anguilla,Cable and Wireless,AIACW,365840,2019-01-01,0.3212,0.0,0.0256,0.0514,60.0,1.0,False,S1
3,Antigua and Barbuda,C & W,ATGCW,344920,2019-01-01,0.3000,0.0,0.0240,0.0480,60.0,1.0,False,S1
4,Argentina,AMX Argentina S A,ARGCM,722310,2020-01-01,0.5200,0.1,0.0800,0.0500,60.0,1.0,True,NaN


In [11]:
# Подготовка P4 Price Plan для Пользователей (без DailyCapLimit)

df_s2_price_plan = df_s1_s2_sponsor_mccmnc\
[['COUNTRY_NAME','NETWORK_NAME','TADIG_CODE','MCCMNC','START_DATE_S2'
  ,'MOC_S2','MTC_S2','SMS_S2','MB_S2','Voice_Increment_SEC_S2','Data_Increment_KB_S2'
  ,'support4G_S2','SPONSOR']].drop_duplicates()

df_s2_price_plan.rename({'COUNTRY_NAME':'Country','NETWORK_NAME':'Operator','TADIG_CODE':'TADIG','START_DATE_S1':'START_DATE',
            'MOC_S2':'MOC','MTC_S2':'MTC','SMS_S2':'SMS','MB_S2':'DATA (MB)',
            'Voice_Increment_SEC_S2':'Voice Increment (SEC)','Data_Increment_KB_S2':'Data Increment (KB)',
            'support4G_S2':'Support 4G'}
           ,axis=1,inplace=True)

df_s2_price_plan.dropna(subset=['MOC','MTC','SMS','DATA (MB)'], how='all', inplace=True)
df_s2_price_plan.MTC.fillna(0,inplace=True)

df_s2_price_plan[['MOC','MTC','SMS','DATA (MB)']] = df_s2_price_plan[['MOC','MTC','SMS','DATA (MB)']]*2

# df_s2_price_plan.loc[df_s2_price_plan.Country=='United States',['MOC','MTC','SMS','DATA (MB)']]\
# = df_s2_price_plan[df_s2_price_plan.Country=='United States'][['MOC','MTC','SMS','DATA (MB)']]*1.25

df_s2_price_plan.to_csv(join(load_folder,'df_s2_price_plan.csv'),index=False)

df_s2_price_plan.head()

,Country,Operator,TADIG,MCCMNC,START_DATE_S2,MOC,MTC,SMS,DATA (MB),Voice Increment (SEC),Data Increment (KB),Support 4G,SPONSOR
8,Australia,Singtel Optus,AUSOP,50590,2020-01-01,0.1265,0.0,0.0184,0.018400,60.0,1.0,True,S1
9,Austria,A1 Telekom Austria AG,AUTMK,23211,2020-01-01,0.0690,0.0,0.0184,0.008050,1.0,1.0,True,S2
10,Austria,A1 Telekom Austria AG,AUTON,23211,2020-01-01,0.0690,0.0,0.0184,0.008050,1.0,1.0,True,S2
11,Austria,A1 Telekom Austria AG,AUTPT,23211,2020-01-01,0.0690,0.0,0.0184,0.008050,1.0,1.0,True,S2
12,Austria,H3G,AUTCA,23210,2020-01-01,0.0460,0.0,0.0069,0.002875,1.0,1.0,False,S2


In [11]:
# Подготовка MB Price Plan для Пользователей

df_s4_price_plan = df_s1_s2_sponsor_mccmnc\
[['COUNTRY_NAME','NETWORK_NAME','TADIG_CODE','MCCMNC','START_DATE_S4'
  ,'MOC_S4','MTC_S4','SMS_S4','MB_S4','DailyCapLimit_S4','Voice_Increment_SEC_S4','Data_Increment_KB_S4'
  ,'support4G_S4','SPONSOR']].drop_duplicates()

df_s4_price_plan.rename({'COUNTRY_NAME':'Country','NETWORK_NAME':'Operator','TADIG_CODE':'TADIG','START_DATE_S4':'START_DATE',
            'MOC_S4':'MOC','MTC_S4':'MTC','SMS_S4':'SMS','MB_S4':'DATA (MB)','DailyCapLimit_S4':'Daily Cap Limit (MB)',
            'Voice_Increment_SEC_S4':'Voice Increment (SEC)','Data_Increment_KB_S4':'Data Increment (KB)',
            'support4G_S4':'Support 4G'}
           ,axis=1,inplace=True)

df_s4_price_plan.dropna(subset=['MOC','MTC','SMS','DATA (MB)'], how='all', inplace=True)
df_s4_price_plan.MTC.fillna(0,inplace=True)

df_s4_price_plan[['MOC','MTC','SMS','DATA (MB)']] = df_s4_price_plan[['MOC','MTC','SMS','DATA (MB)']] * 2

# df_s4_price_plan.loc[df_s4_price_plan.Country=='United States',['MOC','MTC','SMS','DATA (MB)']]\
# = df_s4_price_plan[df_s4_price_plan.Country=='United States'][['MOC','MTC','SMS','DATA (MB)']]*1.25

df_s4_price_plan.sort_values(by='Country').to_csv(join(load_folder,'df_s4_price_plan.csv'),index=False)

df_s4_price_plan.head()

,Country,Operator,TADIG,MCCMNC,START_DATE,MOC,MTC,SMS,DATA (MB),Daily Cap Limit (MB),Voice Increment (SEC),Data Increment (KB),Support 4G,SPONSOR
40,Canada,Rogers Communications Canada Inc,CANRW,302720,2019-07-02,0.520,0.520,0.52,0.010000,NaN,60.0,1.0,True,S1
48,China,China Unicom,CHNCN,46006,2019-04-19,0.520,0.520,0.52,0.012602,NaN,60.0,1.0,True,S1
49,China,China Unicom,CHNCU,46006,2019-04-19,0.520,0.520,0.52,0.012602,NaN,60.0,1.0,True,S1
98,Hong Kong (China),H3G/Hutchinson,HKGH3,45414,2019-04-19,0.026,0.026,0.26,0.006301,NaN,60.0,1.0,True,S1
99,Hong Kong (China),H3G/Hutchinson,HKGHT,45414,2019-04-19,0.026,0.026,0.26,0.006301,NaN,60.0,1.0,True,S1


In [12]:
# Подготовка SMART Price Plan для Пользователей

df_s5_price_plan = df_s1_s2_sponsor_mccmnc\
[['COUNTRY_NAME','NETWORK_NAME','TADIG_CODE','MCCMNC','START_DATE_S5'
  ,'MOC_S5','MTC_S5','SMS_S5','MB_S5','DailyCapLimit_S5','Voice_Increment_SEC_S5','Data_Increment_KB_S5'
  ,'support4G_S5','SPONSOR']].drop_duplicates()

df_s5_price_plan.rename({'COUNTRY_NAME':'Country','NETWORK_NAME':'Operator','TADIG_CODE':'TADIG','START_DATE_S5':'START_DATE',
            'MOC_S5':'MOC','MTC_S5':'MTC','SMS_S5':'SMS','MB_S5':'DATA (MB)','DailyCapLimit_S5':'Daily Cap Limit (MB)',
            'Voice_Increment_SEC_S5':'Voice Increment (SEC)','Data_Increment_KB_S5':'Data Increment (KB)',
            'support4G_S5':'Support 4G'}
           ,axis=1,inplace=True)

df_s5_price_plan.dropna(subset=['MOC','MTC','SMS','DATA (MB)'], how='all', inplace=True)
df_s5_price_plan.MTC.fillna(0,inplace=True)

df_s5_price_plan[['MOC','MTC','SMS','DATA (MB)']] = df_s5_price_plan[['MOC','MTC','SMS','DATA (MB)']] * 2

# df_s5_price_plan.loc[df_s5_price_plan.Country=='United States',['MOC','MTC','SMS','DATA (MB)']]\
# = df_s5_price_plan[df_s5_price_plan.Country=='United States'][['MOC','MTC','SMS','DATA (MB)']]*1.25

df_s5_price_plan.sort_values(by='Country').to_csv(join(load_folder,'df_s5_price_plan.csv'),index=False)

df_s5_price_plan.head()

,Country,Operator,TADIG,MCCMNC,START_DATE,MOC,MTC,SMS,DATA (MB),Daily Cap Limit (MB),Voice Increment (SEC),Data Increment (KB),Support 4G,SPONSOR
1,Albania,Vodafone,ALBVF,27602,2020-01-01,0.322,0.000,0.01725,0.01725,NaN,60.0,1.0,False,S1
4,Argentina,AMX Argentina S A,ARGCM,722310,2020-01-01,0.805,0.069,0.02300,0.01840,NaN,60.0,1.0,True,NaN
5,Argentina,Movistar/Telefonica,ARG01,72207,2020-01-01,1.380,0.230,0.23000,0.23000,NaN,60.0,1.0,False,S1
6,Argentina,Movistar/Telefonica,ARGTM,72207,2020-01-01,1.380,0.230,0.23000,0.23000,NaN,60.0,1.0,False,S1
7,Argentina,Telecom Personal S.A.,ARGTP,72234,2020-01-01,0.575,0.000,0.02300,0.02300,NaN,60.0,1.0,True,NaN


In [12]:
# Подготовка Orange Price Plan для Пользователей (без DailyCapLimit)

df_s6_price_plan = df_s1_s2_sponsor_mccmnc\
[['COUNTRY_NAME','NETWORK_NAME','TADIG_CODE','MCCMNC','START_DATE_S6'
  ,'MOC_S6','MTC_S6','SMS_S6','MB_S6','Voice_Increment_SEC_S6','Data_Increment_KB_S6'
  ,'support4G_S6','SPONSOR']].drop_duplicates()

df_s6_price_plan.rename({'COUNTRY_NAME':'Country','NETWORK_NAME':'Operator','TADIG_CODE':'TADIG','START_DATE_S6':'START_DATE',
            'MOC_S6':'MOC','MTC_S6':'MTC','SMS_S6':'SMS','MB_S6':'DATA (MB)',
            'Voice_Increment_SEC_S6':'Voice Increment (SEC)','Data_Increment_KB_S6':'Data Increment (KB)',
            'support4G_S6':'Support 4G'}
           ,axis=1,inplace=True)

df_s6_price_plan.dropna(subset=['MOC','MTC','SMS','DATA (MB)'], how='all', inplace=True)
df_s6_price_plan.MTC.fillna(0,inplace=True)

df_s6_price_plan[['MOC','MTC','SMS','DATA (MB)']] = df_s6_price_plan[['MOC','MTC','SMS','DATA (MB)']]*2

# df_s6_price_plan.loc[df_s6_price_plan.Country=='United States',['MOC','MTC','SMS','DATA (MB)']]\
# = df_s6_price_plan[df_s6_price_plan.Country=='United States'][['MOC','MTC','SMS','DATA (MB)']]*1.25

df_s6_price_plan.to_csv(join(load_folder,'df_s6_price_plan.csv'),index=False)

df_s6_price_plan.head()

,Country,Operator,TADIG,MCCMNC,START_DATE,MOC,MTC,SMS,DATA (MB),Voice Increment (SEC),Data Increment (KB),Support 4G,SPONSOR
0,Albania,TELEKOM ALBANIA SH A,ALBAM,27601,2020-01-01,0.8970,0.000,0.01495,0.0299,1.0,1.0,True,NaN
1,Albania,Vodafone,ALBVF,27602,2020-01-01,0.4485,0.000,0.01495,0.0299,1.0,1.0,True,S1
2,Anguilla,Cable and Wireless,AIACW,365840,2020-01-01,0.7475,0.000,0.08970,0.3588,1.0,1.0,False,S1
3,Antigua and Barbuda,C & W,ATGCW,344920,2020-01-01,0.7475,0.000,0.08970,0.3588,1.0,1.0,False,S1
4,Argentina,AMX Argentina S A,ARGCM,722310,2020-01-01,0.4186,0.115,0.11960,0.0897,1.0,1.0,False,NaN


In [13]:
# Подготовка JT Price Plan для Пользователей (без DailyCapLimit)

df_s8_price_plan = df_s1_s2_sponsor_mccmnc\
[['COUNTRY_NAME','NETWORK_NAME','TADIG_CODE','MCCMNC','START_DATE_S8'
  ,'MOC_S8','MTC_S8','SMS_S8','MB_S8','Voice_Increment_SEC_S8','Data_Increment_KB_S8'
  ,'support4G_S8','SPONSOR']].drop_duplicates()

df_s8_price_plan.rename({'COUNTRY_NAME':'Country','NETWORK_NAME':'Operator','TADIG_CODE':'TADIG','START_DATE_S8':'START_DATE',
            'MOC_S8':'MOC','MTC_S8':'MTC','SMS_S8':'SMS','MB_S8':'DATA (MB)',
            'Voice_Increment_SEC_S8':'Voice Increment (SEC)','Data_Increment_KB_S8':'Data Increment (KB)',
            'support4G_S8':'Support 4G'}
           ,axis=1,inplace=True)

df_s8_price_plan.dropna(subset=['MOC','MTC','SMS','DATA (MB)'], how='all', inplace=True)
df_s8_price_plan.MTC.fillna(0,inplace=True)

df_s8_price_plan[['MOC','MTC','SMS','DATA (MB)']] = df_s8_price_plan[['MOC','MTC','SMS','DATA (MB)']] * 2

df_s8_price_plan.sort_values(by='Country').to_csv(join(load_folder,'df_s8_price_plan.csv'),index=False)

df_s8_price_plan.head()

,Country,Operator,TADIG,MCCMNC,START_DATE,MOC,MTC,SMS,DATA (MB),Voice Increment (SEC),Data Increment (KB),Support 4G,SPONSOR
0,Albania,TELEKOM ALBANIA SH A,ALBAM,27601,2019-12-01,1.2192,0.0000,0.3864,0.0360,None,1.0,False,NaN
1,Albania,Vodafone,ALBVF,27602,2019-12-01,0.4920,0.0000,0.2040,0.0408,None,1.0,True,S1
2,Anguilla,Cable and Wireless,AIACW,365840,2019-12-01,0.7152,0.0000,0.4776,0.2520,None,1.0,True,S1
3,Antigua and Barbuda,C & W,ATGCW,344920,2019-12-01,0.7152,0.0000,0.4776,0.2520,None,1.0,True,S1
4,Argentina,AMX Argentina S A,ARGCM,722310,2019-12-01,1.7856,0.7152,0.8952,1.0656,None,1.0,True,NaN


### Developments

In [ ]:
df_comb = pd.read_csv(join(downloads, 'CountryMinTariffLte.csv'), usecols=['COUNTRY','SPONSOR COMBINED 2020'])
df_comb.rename({'SPONSOR COMBINED 2020':'SPONSOR'}, axis=1, inplace=True)
df_net = pd.read_csv(join(downloads, 'OperatorMinTariffLte.csv'), usecols=['COUNTRY','NETWORK_ID'])
pd.merge(df_comb, df_net, how='left', on='COUNTRY').to_csv(join(downloads, 'al_network_id.csv'), index=False)

In [24]:
df_al = pd.read_csv(join(load_folder, 'al_network_id.csv'), usecols=['SPONSOR','NETWORK_ID'])
df_al['TADIG_CODE_AL'] = ''
df_al.head()

,SPONSOR,NETWORK_ID,TADIG_CODE_AL
0,S8,5,
1,S8,6,
2,S1,12,
3,S1,10,
4,S1,9,
